# Predicting Location via Indoor Positioning System 
## Analysis and Modeling in Python 
This script and subsequent notebooks are the Python conversion of the Indoor Location case study from the book, 'Data Science in R' by Nolan and Lang. http://rdatasciencecases.org/code.html <BR>
* Team: Jonathan Tan, Lance Dacy, Reannan Mcdaniel, Shawn Jung 
* Last Update: 5/14/2020 

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
#from statsmodels.distributions.empirical_distribution import ECDF
%matplotlib inline 

In [13]:
# Let's load dataframes
offline_pivot = pd.read_csv('./data/offline_pivot.csv')
online_pivot = pd.read_csv('./data/online_pivot.csv')

# re-add pos_x and pos_y for distance error calculation in later stage
offline_pivot[['pos_x', 'pos_y']] = offline_pivot['pos_xy'].str.split('-',n=2, expand=True).astype(float)
online_pivot[['pos_x', 'pos_y']] = online_pivot['pos_xy'].str.split('-',n=2, expand=True).astype(float)

In [27]:
test0 = online_pivot.iloc[0,1:8]
train0 = offline_pivot.iloc[0,1:8]

In [26]:
# the Euclidean distance of first records of each data frame can be calcualted like this. 
np.sqrt(np.sum(np.square((np.array(test0) - np.array(train0)))))

10.412954827804205

In [57]:
def distance(test_array, train_df, number_of_returns=1):
    dist = []
    for i in range(train_df.shape[0]):
        d = np.sqrt(np.sum(np.square((np.array(test_array) - np.array(train_df)[i]))))
        dist.append(d)
    
    dist_result = np.array(dist)
    
    # return top n lowest values

    return dist_result.argsort()[0:number_of_returns][::-1]





In [63]:
test0_pred = distance(test0, offline_pivot.iloc[:,1:8], 1)

print('predicted location:', offline_pivot.iloc[test0_pred]['pos_xy'])
print('original location:', online_pivot.iloc[0]['pos_xy'])

predicted location: 18    1.0-2.0
Name: pos_xy, dtype: object
original location: 0.0-0.05
